### Install requirements using requirements.txt file
#### '$ conda create --name <env> --file <this file>'

Use CUDA for faster computing if Your GPU supports it
It should print something like this if u have cudnn installed
'''
PyTorch version: 2.0.1
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:36:15_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 3080 Ti'''

In [ ]:
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

In [5]:
# imports
import time
from selenium import webdriver
from time import sleep
import cv2
from selenium.webdriver.common import by
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import easyocr
from fuzzywuzzy import fuzz
import os

##### We have to link existing webbrowser session to selenium to bypass twitch.tv security procedures -> twitch.tv won't let you log in to your account (or bot account) and will throw error like this: "Your webbrowser is outdated"

##### In case u don't know how to set up own chromedriver -> https://www.youtube.com/watch?v=Zrx8FSEo9lk

#### Spawn chrome session using command prompt

In [ ]:
os.chdir('C:\Program Files\Google\Chrome\Application') # Path to Google Chrome
os.system(r'chrome.exe --remote-debugging-port=9222 --user-data-dir="link_to_chrome_profile"')

#### Initialize and link selenium webdriver to your already existing webbrowser session

In [7]:
# Initialize options to a chromedriver
opt = Options()
opt.add_experimental_option(name='debuggerAddress', value='localhost:9222')

# Link selenium to existing webbrowser session
service = Service("link_to_chromedriver.exe")
driver = webdriver.Chrome(service=service, options=opt)
# Change url to desired
driver.get(url='desired_url_to_open')

#### Initialize reader object

In [ ]:
# I am using CUDA -> in case you don't want to use CUDA set 'gpu' to 'false'
reader = easyocr.Reader(['pl'], gpu=True)

#### Main script's loop

In [ ]:
# reader bias
ratio = 0
while True:
    # Get screenshot and save to analyze
    driver.get_screenshot_as_file("screenshot.png")
    
    # Read image from which text needs to be extracted
    img = cv2.imread("screenshot.png")
    # Adjust image sector that fits your screen resolution and contains the desired image content
    cropped_img = img[370:500, 680:1200]
    # Read text using model from easyocr
    text = reader.readtext(cropped_img)

    # Validate string value
    try:
        bbox, value, score = text[0]
        ratio = fuzz.ratio('text_to_validate', value)
        print(f'Value: {ratio}')
        
        # Determine bias empirically
        if ratio > 50:
            # Find input element and fill it with command, clear element
            # chat-wysiwyg-input__editor is a class name of the input element on twitch chat  
            inputElement = driver.find_element(by=by.By.CLASS_NAME, value='chat-wysiwyg-input__editor')
            inputElement.send_keys(Keys.CLEAR)
            # Send your command
            inputElement.send_keys(',boss deaths +1')
            inputElement.send_keys(Keys.ENTER)
            # This print is strictly for debugging purposes
            print('FOUND')
    except:
        # This print is strictly for debugging purposes
        print('NOT FOUND')
        continue
    # Adjust sleep time between readings
    time.sleep(5)

driver.quit()